# Colab for Feature Engineering.  #

---


## To Do: ##
1. Separate the Duplicates and Non-Duplicates
2. Copy the DataFrames into new variables
3. Process the data into the feature to be extracted
4. Plot the Histogram to see to observe the distribution


---


##Features##

1. Number of unique words which occur in q1 and q2 
2. Ratio of common words / total words (q1+q2)
2. Common Word Ratio min ( words common/ min(len(q1), len(q2)))
2. Common Word Ratio mmax ( words common/ max(len(q1), len(q2)))
2. Common Stop Words min ( common stopwords/ min(len(q1), len(q2)))
2. Common Stop Words max  ( common stopwords/ max(len(q1), len(q2)))
2. Common Tokens min ( common Tokens / min(len(q1), len(q2)))
2. Common Tokens max  ( common Tokens / max(len(q1), len(q2)))
2. Common Adjectives min ( common adjectives /min(len(q1), len(q2)))
2. Common Adjectives max ( common adjectives /max(len(q1), len(q2)))
2. Common Noun min ( common nouns / min(len(q1), len(q2)))
2. Common Noun max ( common nouns / max(len(q1), len(q2)))
2. Fuzz ratio
2. Fuzz partial ratio 
2. Fuzz Token Sort Ratio 
2. Fuzz Token Set Ratio
2. Mean Length of 2 questions
2. Ratio of Length of Questions ( len(q1) / len(q2) )
2. Absolute Length Difference (| len(q1) - len(q2) |
2. Longest Matching Substring min ( longest substring/min(len(q1), len(q2)))
2. Longest Matching Substring max ( longest substring/max(len(q1), len(q2)))



Download your required libraries here

In [ ]:
!pip install bs4
!pip install fuzzywuzzy
!pip install TextBlob
!python -m spacy download en_core_web_lg

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Python38\python.exe -m pip install --upgrade pip' command.


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')

You should consider upgrading via the 'C:\Python38\python.exe -m pip install --upgrade pip' command.



[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


Import your required libraries here

In [ ]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from fuzzywuzzy import fuzz
from difflib import SequenceMatcher #For finding longest substring
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import en_core_web_lg
nlp = spacy.load('en_core_web_lg')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger') # for pos tagging


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luciu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luciu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\luciu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Mounting the dataset onto this google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/CS3244 45 Project/quora-question-pairs/"
train_set = pd.read_csv('./train.csv')
test_set = pd.read_csv('./test.csv')

In [ ]:
%cd './Desktop/CS3244/Project/quora-question-pairs'

In [ ]:
# train_set = pd.read_csv('train.csv')

train_set = pd.read_csv('./Desktop/NUS Lecture Notes/Y3S1/CS3244/Project/train.csv')

##**Dropping NA Rows**

In [ ]:
train_set.dropna(inplace=True)

In [ ]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404287 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404287 non-null  int64 
 1   qid1          404287 non-null  int64 
 2   qid2          404287 non-null  int64 
 3   question1     404287 non-null  object
 4   question2     404287 non-null  object
 5   is_duplicate  404287 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 21.6+ MB


##**Preprocess the questions**

In [ ]:
# This function accepts a question and preprocesses it. Returns cleaned question.
def preprocess(q):
  # Firstly, we convert to lowercase and remove trailing and leading spaces
  q = str(q).lower().strip()

  # Replace certain special characters with their string equivalents
  q = q.replace('%', ' percent')
  q = q.replace('$', ' dollar ')
  q = q.replace('₹', ' rupee ')
  q = q.replace('€', ' euro ')
  q = q.replace('@', ' at ')

  # The pattern '[math]' appears around 900 times in the whole dataset.
  q = q.replace('[math]', '')

  # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
  q = q.replace(',000,000,000 ', 'b ')
  q = q.replace(',000,000 ', 'm ')
  q = q.replace(',000 ', 'k ')
  q = re.sub(r'([0-9]+)000000000', r'\1b', q)
  q = re.sub(r'([0-9]+)000000', r'\1m', q)
  q = re.sub(r'([0-9]+)000', r'\1k', q)

  # Decontracting words
  # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
  # https://stackoverflow.com/a/19794953
  contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
  }

  q_decontracted = []

  for word in q.split():
    if word in contractions:
      word = contractions[word]
  
    q_decontracted.append(word)

  q = ' '.join(q_decontracted)
  q = q.replace("'ve", " have")
  q = q.replace("n't", " not")
  q = q.replace("'re", " are")
  q = q.replace("'ll", " will")

  # Removing HTML tags
  q = BeautifulSoup(q)
  q = q.get_text()

  # Remove punctuations
  pattern = re.compile('\W')
  q = re.sub(pattern, ' ', q).strip()

  return q

In [ ]:
train_set['question1'] = train_set['question1'].apply(preprocess)
train_set['question2'] = train_set['question2'].apply(preprocess)

c:\python38\lib\site-packages\bs4\__init__.py:332: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


##**Removing empty questions**

In [ ]:
train_set = train_set.drop(train_set[train_set['question1'] == ''].index)
train_set = train_set.drop(train_set[train_set['question2'] == ''].index)

##**Jun An**

1. Ratio of Common Words (Common words / total words) (Done)
2. Ratio of Common Tokens (Common tokens/ max(q1, q2)) (Done)
3. Fuzz partial ratio (Done)
4. Longest Matching Substring Min (Done)

In [ ]:
def num_common_words_ratio(row):
  set1 = set(row['question1'].lower().split())
  set2 = set(row['question2'].lower().split())
  total = len(set1) + len(set2)
  return len(set1.intersection(set2))/total

In [ ]:
train_set['common_words_ratio'] = train_set.apply(num_common_words_ratio, axis=1)

In [ ]:
def common_tokens_ratio_max(row):
  q1 = set(word_tokenize(row['question1'].lower()))
  q2 = set(word_tokenize(row['question2'].lower()))
  stop_words = set(stopwords.words('english'))
  token1 = [word for word in q1 if word not in stop_words]
  token2 = [word for word in q2 if word not in stop_words]
  ratio = len(set(token1).intersection(set(token2))) / max(len(row['question1']), len(row['question2']))

  return ratio


In [ ]:
train_set['common_tokens_ratio'] = train_set.apply(common_tokens_ratio_max, axis=1)

In [ ]:
def fuzz_partial_ratio(row):
  q1 = row['question1']
  q2 = row['question2']
  fuzz_partial = fuzz.partial_ratio(q1,q2)
  return fuzz_partial

In [ ]:
train_set['fuzz_partial_ratio'] = train_set.apply(fuzz_partial_ratio, axis=1)

In [ ]:
def min_longest_substring(row):
  q1 = row['question1']
  q2 = row['question2']
  match = SequenceMatcher(None, q1, q2).find_longest_match(0, len(q1), 0, len(q2))
  return match.size/min(len(q1), len(q2))

In [ ]:
train_set['min_longest_substring'] = train_set.apply(min_longest_substring, axis=1)

In [ ]:
train_set.head()

,id,qid1,qid2,question1,question2,is_duplicate,common_words_ratio,common_tokens_ratio,fuzz_partial_ratio,min_longest_substring
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,0.478261,0.076923,100,1.000000
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,0.291667,0.045977,74,0.600000
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,0.166667,0.027778,46,0.172414
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when 23 24 math is divi...,0,0.000000,0.000000,11,0.040816
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,0.200000,0.026667,55,0.157895


##**Penn Han**

1. Number of unique words that occur in q1 and q2
2. Ratio of Common Tokens to min(len(q1), len(q2))
3. Fuzz Ratio
4. Absolute Length Difference between q1 and q2
5. Mean TF-IDF value
6. Mean IDF-weighted vector

In [ ]:
def unique_words_count(row):
  set1 = set(row['question1'].lower().split())
  set2 = set(row['question2'].lower().split())
  return len(set1.intersection(set2))

In [ ]:
train_set["unique_words_count"] = train_set.apply(unique_words_count, axis=1)

In [ ]:
def common_token_ratio_min(row):
  q1 = set(word_tokenize(row['question1'].lower()))
  q2 = set(word_tokenize(row['question2'].lower()))
  stop_words = set(stopwords.words('english'))
  token1 = [word for word in q1 if word not in stop_words]
  token2 = [word for word in q2 if word not in stop_words]
  ratio = len(set(token1).intersection(set(token2))) / min(len(row['question1']), len(row['question2']))
  return ratio

In [ ]:
train_set["common_token_ratio_min"] = train_set.apply(common_token_ratio_min, axis=1)

In [ ]:
def fuzz_ratio(row):
  q1 = row['question1']
  q2 = row['question2']
  fuzz_ratio = fuzz.ratio(q1,q2)
  return fuzz_ratio

In [ ]:
train_set["fuzz_ratio"] = train_set.apply(fuzz_ratio, axis=1)

In [ ]:
def abs_len_difference(row):
  q1 = row['question1']
  q2 = row['question2']
  abs_len_diff = abs(len(q1) - len(q2))
  return abs_len_diff

In [ ]:
train_set["abs_len_difference"] = train_set.apply(abs_len_difference, axis=1)

In [ ]:
#Stop words not removed PLEASE ONLY USE EITHER THIS OR THE BELOW, NOT BOTH

tf_idf_vectoriser = TfidfVectorizer(lowercase=True)
q1_train_list = list(train_set['question1'])
q2_train_list = list(train_set['question2'])
question_corpus = list(q1_train_list + q2_train_list)
tf_idf_vectoriser.fit(question_corpus)
idf = dict(zip(tf_idf_vectoriser.get_feature_names(), tf_idf_vectoriser.idf_))  #For Weighted W2V
nlp = en_core_web_lg.load()

In [ ]:
#Stop words removed  PLEASE ONLY USE THIS OR THE ABOVE, NOT BOTH

tf_idf_vectoriser = TfidfVectorizer(lowercase=True)
stop_words = set(stopwords.words('english'))
question_corpus = []
for question1, question2 in zip(list(train_set['question1']), list(train_set['question2'])):
  q1 = word_tokenize(question1.lower())  
  token1 = " ".join([word for word in q1 if word not in stop_words])
  q2 = word_tokenize(question2.lower())
  token2 = " ".join([word for word in q2 if word not in stop_words])
  question_corpus.append((token1 + token2))
tf_idf_vectoriser.fit(question_corpus)
idf = dict(zip(tf_idf_vectoriser.get_feature_names(), tf_idf_vectoriser.idf_))  #For Weighted W2V
nlp = en_core_web_lg.load()

In [ ]:
def mean_tfidf_value_q1(row):
  q1 = word_tokenize(row['question1'].lower())
  stop_words = set(stopwords.words('english'))
  token1 = [word for word in q1 if word not in stop_words]
  if len(token1) > 0:
    q1_vector_matrix = tf_idf_vectoriser.transform(token1)  #Transform must take in a iterable so [str]
    return q1_vector_matrix  #Returns a sparse matrix
  else:
    return 0

In [ ]:
def mean_tfidf_value_q2(row):
  q2 = set(word_tokenize(row['question2'].lower()))
  stop_words = set(stopwords.words('english'))
  token2 = [word for word in q1 if word not in stop_words]
  if len(token1) > 0:
    q2_vector_matrix = tf_idf_vectoriser.transform(token2)  #Transform must take in a iterable so [str]
    return q2_vector_matrix  #Returns a sparse matrix
  else:
    return 0

In [ ]:
def calculate_weighted_vector(question):
    weighted_vectors = []
    doc = nlp(question)
    mean_vec = np.zeros((len(doc[0].vector)))
    for word in doc:
        vector = word.vector
        if str(word) in idf:
            idf_weight = idf[str(word)]
        else:
            idf_weight = 0
        mean_vec += vector * idf_weight
    mean_vec /= len(doc)
    return mean_vec

In [ ]:
def mean_idfweighted_vector_q1(row):
  idfweighted_vector_q1 = calculate_weighted_vector(row['question1'])
  return idfweighted_vector_q1

In [ ]:
def mean_idfweighted_vector_q2(row):
  idfweighted_vector_q2 = calculate_weighted_vector(row['question2'])
  return idfweighted_vector_q2

In [ ]:
train_set["tfidf_matrix_q1"] = train_set.apply(mean_tfidf_value_q1, axis=1)
train_set["tfidf_matrix_q2"] = train_set.apply(mean_tfidf_value_q2, axis=1)
train_set["mean_idfweighted_vector_q1"] = train_set.apply(mean_idfweighted_vector_q1, axis=1)
train_set["mean_idfweighted_vector_q2"] = train_set.apply(mean_idfweighted_vector_q2, axis=1)

In [ ]:
train_set.head()

,id,qid1,qid2,question1,question2,is_duplicate,common_words_ratio,common_tokens_ratio,fuzz_partial_ratio,min_longest_substring,unique_words_count,common_token_ratio_min,fuzz_ratio,abs_len_difference,tfidf_matrix_q1,tfidf_matrix_q2,mean_idfweighted_vector_q1,mean_idfweighted_vector_q2
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,0.478261,0.076923,100,1.000000,11,0.089286,93,9,"(0, 265917)\t1.0\n (1, 265917)\t1.0\n (2, ...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[-0.28463623353413176, 0.9678521794932229, 0.2...","[-0.017922272284825642, 0.9931385070085526, 0...."
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,0.291667,0.045977,74,0.600000,7,0.080000,66,37,"(0, 267012)\t1.0\n (1, 157150)\t1.0\n (2, ...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[0.7488621671994528, 0.7630542020003, 1.556344...","[0.19233562227557688, 0.420634108431199, 1.343..."
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,0.166667,0.027778,46,0.172414,4,0.034483,43,14,"(0, 140422)\t1.0\n (1, 262303)\t1.0\n (2, ...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[0.07783552578517369, 0.9335091284343174, -0.1...","[-1.574381485581398, 1.3970335014164448, -0.48..."
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when 23 24 math is divi...,0,0.000000,0.000000,11,0.040816,0,0.000000,9,9,"(0, 179734)\t1.0\n (1, 167642)\t1.0\n (2, ...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[-0.0391580859820048, 0.38401350875695545, -0....","[-0.024673760930697123, 1.936501924196879, 1.7..."
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,0.200000,0.026667,55,0.157895,4,0.052632,35,37,"(0, 199549)\t1.0\n (1, 84686)\t1.0\n (2, 3...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[-0.7563397467136384, 1.2002705613772073, -0.3...","[-0.8987145317452294, 0.07580995985439845, 0.1..."


## Jeremy
1. common stop words min
2. common noun min
3. mean length of 2 questions

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
def get_min_len_qn(row):
    return min(len(row['question1'].split()), len(row['question2'].split()))

In [ ]:
def calc_common_stop_words_min(row):
    q1 = word_tokenize(row['question1'])
    q2 = word_tokenize(row['question2'])
    stop_words_q1 = set([x for x in q1 if x in stop_words])
    stop_words_q2 = set([x for x in q2 if x in stop_words])
    num_intersect = len(stop_words_q1.intersection(stop_words_q2))
    return num_intersect / get_min_len_qn(row)

In [ ]:
train_set['common_stop_words_min'] = train_set.apply(calc_common_stop_words_min, axis=1)

In [ ]:
def calc_common_nouns_min(row):
    q1_tokens = word_tokenize(row["question1"].lower())
    q2_tokens = word_tokenize(row["question2"].lower())
    pos_tagged_q1 = nltk.pos_tag(q1_tokens)
    pos_tagged_q2 = nltk.pos_tag(q2_tokens)
    # x[0] is the word, x[1] is the tag
    q1_nouns = set([x[0] for x in pos_tagged_q1 if x[1] == "NN"]) 
    q2_nouns = set([x[0] for x in pos_tagged_q2 if x[1] == "NN"])
    return len(q1_nouns.intersection(q2_nouns)) / get_min_len_qn(row)

In [ ]:
train_set['common_nouns_min'] = train_set.apply(calc_common_nouns_min, axis=1)

In [ ]:
def mean_len_qns(row):
    return (len(word_tokenize(row["question1"].lower())) + len(word_tokenize(row["question2"].lower()))) / 2

In [ ]:
train_set['mean_len'] = train_set.apply(mean_len_qns, axis=1)

In [ ]:
train_set.head(5)

,id,qid1,qid2,question1,question2,is_duplicate,common_words_ratio,common_tokens_ratio,fuzz_partial_ratio,min_longest_substring,...,common_token_ratio_min,fuzz_ratio,abs_len_difference,tfidf_matrix_q1,tfidf_matrix_q2,mean_idfweighted_vector_q1,mean_idfweighted_vector_q2,common_stop_words_min,common_nouns_min,mean_len
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,0.478261,0.076923,100,1.000000,...,0.089286,93,9,"(0, 265917)\t1.0\n (1, 265917)\t1.0\n (2, ...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[-0.28463623353413176, 0.9678521794932229, 0.2...","[-0.017922272284825642, 0.9931385070085526, 0....",0.500000,0.250000,13.0
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,0.291667,0.045977,74,0.600000,...,0.080000,66,37,"(0, 267012)\t1.0\n (1, 157150)\t1.0\n (2, ...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[0.7488621671994528, 0.7630542020003, 1.556344...","[0.19233562227557688, 0.420634108431199, 1.343...",0.300000,0.300000,12.5
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,0.166667,0.027778,46,0.172414,...,0.034483,43,14,"(0, 140422)\t1.0\n (1, 262303)\t1.0\n (2, ...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[0.07783552578517369, 0.9335091284343174, -0.1...","[-1.574381485581398, 1.3970335014164448, -0.48...",0.200000,0.100000,12.0
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when 23 24 math is divi...,0,0.000000,0.000000,11,0.040816,...,0.000000,9,9,"(0, 179734)\t1.0\n (1, 167642)\t1.0\n (2, ...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[-0.0391580859820048, 0.38401350875695545, -0....","[-0.024673760930697123, 1.936501924196879, 1.7...",0.000000,0.000000,11.5
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,0.200000,0.026667,55,0.157895,...,0.052632,35,37,"(0, 199549)\t1.0\n (1, 84686)\t1.0\n (2, 3...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[-0.7563397467136384, 1.2002705613772073, -0.3...","[-0.8987145317452294, 0.07580995985439845, 0.1...",0.285714,0.285714,10.0


##Kay Chi
1. Common stop words max
2. Common noun max
3. Ratio of length of questions

In [ ]:
def get_max_len_qn(row):
    return max(len(row['question1'].split()), len(row['question2'].split()))

In [ ]:
def calc_common_stop_words_max(row):
    q1 = word_tokenize(row['question1'])
    q2 = word_tokenize(row['question2'])
    stop_words_q1 = set([x for x in q1 if x in stop_words])
    stop_words_q2 = set([x for x in q2 if x in stop_words])
    num_intersect = len(stop_words_q1.intersection(stop_words_q2))
    return num_intersect / get_max_len_qn(row)

In [ ]:
train_set['common_stop_words_max'] = train_set.apply(calc_common_stop_words_max, axis=1)

In [ ]:
def calc_common_nouns_max(row):
    q1_tokens = word_tokenize(row["question1"].lower())
    q2_tokens = word_tokenize(row["question2"].lower())
    pos_tagged_q1 = nltk.pos_tag(q1_tokens)
    pos_tagged_q2 = nltk.pos_tag(q2_tokens)
    # x[0] is the word, x[1] is the tag
    q1_nouns = set([x[0] for x in pos_tagged_q1 if x[1] == "NN"]) 
    q2_nouns = set([x[0] for x in pos_tagged_q2 if x[1] == "NN"])
    return len(q1_nouns.intersection(q2_nouns)) / get_max_len_qn(row)

In [ ]:
train_set['common_nouns_max'] = train_set.apply(calc_common_nouns_max, axis=1)

In [ ]:
def ratio_len_qn(row):
  q1 = row['question1']
  q2 = row['question2']
  return len(q1) / len(q2)

In [ ]:
train_set['ratio_len_qn'] = train_set.apply(ratio_len_qn, axis=1)

In [ ]:
train_set.head()

,id,qid1,qid2,question1,question2,is_duplicate,common_words_ratio,common_tokens_ratio,fuzz_partial_ratio,min_longest_substring,...,tfidf_matrix_q1,tfidf_matrix_q2,mean_idfweighted_vector_q1,mean_idfweighted_vector_q2,common_stop_words_min,common_nouns_min,mean_len,common_stop_words_max,common_nouns_max,ratio_len_qn
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,0.478261,0.076923,100,1.000000,...,"(0, 265917)\t1.0\n (1, 265917)\t1.0\n (2, ...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[-0.28463623353413176, 0.9678521794932229, 0.2...","[-0.017922272284825642, 0.9931385070085526, 0....",0.500000,0.250000,13.0,0.428571,0.214286,1.160714
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,0.291667,0.045977,74,0.600000,...,"(0, 267012)\t1.0\n (1, 157150)\t1.0\n (2, ...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[0.7488621671994528, 0.7630542020003, 1.556344...","[0.19233562227557688, 0.420634108431199, 1.343...",0.300000,0.300000,12.5,0.200000,0.200000,0.574713
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,0.166667,0.027778,46,0.172414,...,"(0, 140422)\t1.0\n (1, 262303)\t1.0\n (2, ...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[0.07783552578517369, 0.9335091284343174, -0.1...","[-1.574381485581398, 1.3970335014164448, -0.48...",0.200000,0.100000,12.0,0.142857,0.071429,1.241379
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when 23 24 math is divi...,0,0.000000,0.000000,11,0.040816,...,"(0, 179734)\t1.0\n (1, 167642)\t1.0\n (2, ...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[-0.0391580859820048, 0.38401350875695545, -0....","[-0.024673760930697123, 1.936501924196879, 1.7...",0.000000,0.000000,11.5,0.000000,0.000000,0.844828
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,0.200000,0.026667,55,0.157895,...,"(0, 199549)\t1.0\n (1, 84686)\t1.0\n (2, 3...","(0, 164246)\t1.0\n (1, 251513)\t1.0\n (2, ...","[-0.7563397467136384, 1.2002705613772073, -0.3...","[-0.8987145317452294, 0.07580995985439845, 0.1...",0.285714,0.285714,10.0,0.153846,0.153846,1.973684


## YS
1. Common Word Ratio max ( words common/ max(len(q1), len(q2))) 
2. Common Adjectives max ( common adjectives /max(len(q1), len(q2)))
3. Fuzz Token Set Ratio 

In [ ]:
def common_word_ratio_max(row):
  q1 = row['question1']
  q2 = row['question2']
  return len(set(q1).intersection(set(q2))) / max(len(q1), len(q2))

train_set['common_word_ratio_max'] = train_set.apply(common_word_ratio_max, axis=1)

In [ ]:
# This has been tested to be correct, but result seems off.
def get_adjectives(text):
  blob = TextBlob(text)
  return set(word for (word,tag) in blob.tags if tag.startswith("JJ"))
  
def common_adjectives_max(row):
  q1 = row['question1']
  q2 = row['question2']
  return len(get_adjectives(q1).intersection(get_adjectives(q2))) / max(len(q1), len(q2))

train_set['common_adjectives_max'] = train_set.apply(common_adjectives_max, axis=1)

In [ ]:
def calc_fuzz_token_set_ratio(row):
  q1 = row['question1']
  q2 = row['question2']
  return fuzz.token_set_ratio(q1, q2)

train_set['fuzz_token_set_ratio'] = train_set.apply(calc_fuzz_token_set_ratio, axis=1)

In [ ]:
train_set.head()

,id,qid1,qid2,question1,question2,is_duplicate,common_words_ratio,common_tokens_ratio,fuzz_partial_ratio,min_longest_substring,...,mean_idfweighted_vector_q2,common_stop_words_min,common_nouns_min,mean_len,common_stop_words_max,common_nouns_max,ratio_len_qn,common_word_ratio_max,common_adjectives_max,fuzz_token_set_ratio
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,0.478261,0.076923,100,1.000000,...,"[-0.017922272284825642, 0.9931385070085526, 0....",0.500000,0.250000,13.0,0.428571,0.214286,1.160714,0.307692,0.0,100
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,0.291667,0.045977,74,0.600000,...,"[0.19233562227557688, 0.420634108431199, 1.343...",0.300000,0.300000,12.5,0.200000,0.200000,0.574713,0.172414,0.0,86
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,0.166667,0.027778,46,0.172414,...,"[-1.574381485581398, 1.3970335014164448, -0.48...",0.200000,0.100000,12.0,0.142857,0.071429,1.241379,0.236111,0.0,63
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when 23 24 math is divi...,0,0.000000,0.000000,11,0.040816,...,"[-0.024673760930697123, 1.936501924196879, 1.7...",0.000000,0.000000,11.5,0.000000,0.000000,0.844828,0.224138,0.0,28
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,0.200000,0.026667,55,0.157895,...,"[-0.8987145317452294, 0.07580995985439845, 0.1...",0.285714,0.285714,10.0,0.153846,0.153846,1.973684,0.213333,0.0,67


##**Neaton**

In [ ]:
def common_words_ratio_min(row):
  set1 = set(row['question1'].lower().split())
  set2 = set(row['question2'].lower().split())
  common_words = len(set1.intersection(set2))
  return common_words/min(len(set1), len(set2))


In [ ]:
train_set['common_words_ratio_min'] = train_set.apply(common_words_ratio_min, axis=1)

In [ ]:
# This has been tested to be correct, but result seems off.
def get_adjectives(text):
  blob = TextBlob(text)
  return set(word for (word,tag) in blob.tags if tag.startswith("JJ"))
  
def common_adjectives_min(row):
  q1 = row['question1']
  q2 = row['question2']
  return len(get_adjectives(q1).intersection(get_adjectives(q2))) / min(len(q1), len(q2))

In [ ]:
train_set['common_adjectives_min'] = train_set.apply(common_adjectives_min, axis=1)

In [ ]:
def fuzz_token_sort_ratio(row):
  q1 = row['question1']
  q2 = row['question2']
  fuzz_token = fuzz.token_sort_ratio(q1,q2)
  return fuzz_token

In [ ]:
train_set['fuzz_token_sort_ratio'] = train_set.apply(fuzz_token_sort_ratio, axis=1)

In [ ]:
def max_longest_substring(row):
  q1 = row['question1']
  q2 = row['question2']
  match = SequenceMatcher(None, q1, q2).find_longest_match(0, len(q1), 0, len(q2))
  return match.size/max(len(q1), len(q2))

In [ ]:
train_set['max_longest_substring'] = train_set.apply(max_longest_substring, axis=1)

In [ ]:
train_set.head()

,id,qid1,qid2,question1,question2,is_duplicate,common_words_ratio,common_tokens_ratio,fuzz_partial_ratio,min_longest_substring,...,common_stop_words_max,common_nouns_max,ratio_len_qn,common_word_ratio_max,common_adjectives_max,fuzz_token_set_ratio,common_words_ratio_min,common_adjectives_min,fuzz_token_sort_ratio,max_longest_substring
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,0.478261,0.076923,100,1.000000,...,0.428571,0.214286,1.160714,0.307692,0.0,100,1.000000,0.0,93,0.861538
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,0.291667,0.045977,74,0.600000,...,0.200000,0.200000,0.574713,0.172414,0.0,86,0.700000,0.0,63,0.344828
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,0.166667,0.027778,46,0.172414,...,0.142857,0.071429,1.241379,0.236111,0.0,63,0.400000,0.0,63,0.138889
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when 23 24 math is divi...,0,0.000000,0.000000,11,0.040816,...,0.000000,0.000000,0.844828,0.224138,0.0,28,0.000000,0.0,25,0.034483
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,0.200000,0.026667,55,0.157895,...,0.153846,0.153846,1.973684,0.213333,0.0,67,0.571429,0.0,47,0.080000


In [ ]:
train_set.to_csv('train_set_with_features.csv')

In [ ]:
train_set["tfidf_matrix_q1"]

0           (0, 265917)\t1.0\n  (1, 265917)\t1.0\n  (2, ...
1           (0, 267012)\t1.0\n  (1, 157150)\t1.0\n  (2, ...
2           (0, 140422)\t1.0\n  (1, 262303)\t1.0\n  (2, ...
3           (0, 179734)\t1.0\n  (1, 167642)\t1.0\n  (2, ...
4           (0, 199549)\t1.0\n  (1, 84686)\t1.0\n  (2, 3...
                                ...                        
404285      (0, 173304)\t1.0\n  (1, 155186)\t1.0\n  (2, ...
404286      (0, 34288)\t1.0\n  (1, 162979)\t1.0\n  (2, 7...
404287                  (0, 199549)\t1.0\n  (1, 59341)\t1.0
404288      (0, 23149)\t1.0\n  (1, 20462)\t1.0\n  (2, 67...
404289      (0, 164246)\t1.0\n  (1, 251513)\t1.0\n  (2, ...
Name: tfidf_matrix_q1, Length: 404266, dtype: object

In [ ]:
train_set["mean_idfweighted_vector_q1"]

0         [-0.28463623353413176, 0.9678521794932229, 0.2...
1         [0.7488621671994528, 0.7630542020003, 1.556344...
2         [0.07783552578517369, 0.9335091284343174, -0.1...
3         [-0.0391580859820048, 0.38401350875695545, -0....
4         [-0.7563397467136384, 1.2002705613772073, -0.3...
                                ...                        
404285    [-0.3489754008395331, -0.16148637341601507, 0....
404286    [-0.38235482573509216, 0.9946302771568298, -0....
404287    [-0.008712098002433777, -0.23649294674396515, ...
404288    [-0.021404032905896504, 0.6523470787538422, 0....
404289    [-0.586537778377533, -0.2981147915124893, -0.9...
Name: mean_idfweighted_vector_q1, Length: 404266, dtype: object

In [ ]:
train_set2 = train_set.drop(columns=["tfidf_matrix_q1", "tfidf_matrix_q2", "mean_idfweighted_vector_q1", "mean_idfweighted_vector_q2"])

In [ ]:
train_set2.to_csv("train_set_2.csv")